__Creating Different Movie Barcodes__

In the last report, we went through the basic steps of creating a movie barcode in detail. One of those steps was creating a User Defined Function(UDF) that extracted a "Representative Colour" from each frame. The UDF we had used in the previous report was a function that extracted the value of the pixel located in the very middle of each frame. However, it is possible to create different kinds of movie barcodes with a single video by changing which colour you choose as your representative color. In this report we will learn how to create 5 different kinds of movie barcodes including the barcode created in the previous report.

__0. Review__

In [4]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import imageio

import random

The tools we had imported last time will all be needed again. In addition, we will also need to import the "random" tool to make certain barcodes.

In [ ]:
filename='C:\\Users\\Luke\\Desktop\\Frozen 2 Teaser.mp4'
cap = cv2.VideoCapture(filename) 

CList = []

The code above opens the selected videofile in Python, and the CList is created to provide a list to add all of the representative colors.

In [ ]:
while(True):
    ret, frame = cap.read()

    if ret == False:
        break 

    c = getRepColor(frame)
    CList.append(c)

This is the loop that reads each frame of the previously selected video and adds each extracted representative color to the CList. The UDF getRepColor(frame) will be created later on in the report.

In [ ]:
img = np.zeros((1000,len(CList),3)) 

for i in range(len(CList)):
    img[:,i] = CList[i]

This code creates the barcode by stretching out each representative color in order.

In [ ]:
#b, g, r = cv2.split(img)
#img2 = cv2.merge([r,g,b])

img2 = img.astype(np.uint8)
#plt.imshow(img2)
#plt.show()
imageio.imwrite('MovieBarcode(Center)-Frozen2Teaser.png', img2)

cv2.destroyAllWindows()

This time we won't be showing the barcode using plt and will directly save the barcode as an imagefile. Therefore we won't have to change the order of the r,g,b values. Save the barcode with whichever name you prefer.

Now let's create the UDFs.

__1. Extracting the Center Pixel__

In [1]:
def getRepColor(img):
    rcenter = img.shape[0]//2
    ccenter = img.shape[1]//2
    color = img[rcenter, ccenter]
    return color

As mentioned in the previous report, img.shape returns the height, width, and depth of an image. Therefore, img.shape[0] is the height, img.shape[1] is the width, and img.shape[2] is the depth. 'rcenter' and 'ccenter' each represent the middle value of the hight and width, so 'color' is the color of the pixel in the center of the frame.

![image](MovieBarcode-Frozen2Teaser(Center).png)

This is the movie barcode of the 2 minute teaser trailer of the movie Frozen 2.


__2. Extracting a Random Pixel__

In [2]:
def getRepColor(img):
    rrandom = random.randint(1,img.shape[0])
    crandom = random.randint(1,img.shape[1])
    color = img[rrandom, crandom]
    return color

This is where we need to use the random tool we had imported earlier. 'random.randint(range)' chooses any random integer within the selected range. Therefore rrandom and crandom each represent random integers between 1 to the length of the height, and 1 to the length of the width. Therefore, color returns the r,g,b values of any random pixel within the frame.

![image](MovieBarcode-Frozen2Teaser(Ran).png)

__3. Finding the Average of a Number of Random Pixels__

In [3]:
#1
def getRepColor(img):
    RList=[]
    rsum=0; gsum=0; bsum=0
    #2
    for i in range(100):
        rran = random.randint(1,img.shape[0]-1)
        cran = random.randint(1,img.shape[1]-1)
        RanPixel=img[rran,cran]
        RList.append(RanPixel)
    #3    
    for i in RList:
        rsum += i[0]
        gsum += i[1]
        bsum += i[2]
    #4    
    rmean = rsum//100
    gmean = gsum//100
    bmean = bsum//100
    color = [rmean, gmean, bmean]     
    return color

First of all, create an RList that will add all of the selected pixels. Then, give the value 0 to rsum, gsum, and bsum. Each of these sums will be used to add the total sum of the red, green, and blue values in the RList.

The first loop decides how many pixels will be randomly chosen from each frame. Due to the slow processing speed of my computer, I have only extracted 100 pixels per frame, but in order to get a reliable result it is recommended to extract at least 10 percent of all pixels in each frame. The method used to choose the random pixels are the same as what we used previously.

'i[0]', 'i[1]', and 'i[2]' each represent the red, green, and blue values of each representative color. Therefore this loop adds all of the red, green, and blue values into the sum.

Divide the sum with the amount of pixels you had extracted, and you would get the average value of each color. Give this value to 'color' and return color.

![image](MovieBarcode-Frozen2Teaser(RanAvg).png)

Note that the color sequence of this barcode is similar to the random barcode, but the drastic color change is smoothened out.

__4. Returning the Average of Every Pixel__

In [1]:
def getRepColor(img):
    #1
    RList=[]
    rsum=0; gsum=0; bsum=0
    a=0; b=0
    Total_Pixels=img.shape[0]*img.shape[1] 
    #2
    for a in range(img.shape[0]):
        for b in range(img.shape[1]):
            Pixel = img[a,b]
            RList.append(Pixel)          
    #3
    for i in RList:
        rsum += i[0]
        gsum += i[1]
        bsum += i[2]
    rmean = rsum//Total_Pixels
    gmean = gsum//Total_Pixels
    bmean = bsum//Total_Pixels
    color = [rmean, gmean, bmean] 
    return color

Step1 is the same step as the first step we went through when finding the random average.

Step2 allows you to add every single pixel within each frame into the RList by using a nested in for loop.

Step3 is the same as the fourth step of finding the random average. However, this time we divide the sum with 'Total_Pixels', which is the total amount of pixels in a frame since we are adding the value of every frame.

![image](MovieBarcode-Wild Flower(TotalAvg2).png)

From now on we will be using the 21 second teaser video of the music video 'Wild Flower' to reduce the amount of data being processed.

__5. Extracting the Most Frequently Used Pixel__

In [2]:
def getRepColor(img):
    #1
    Neo=np.zeros((256,256,256))
    #2
    for a in range(img.shape[0]):
        for b in range(img.shape[1]):
            Pixel = img[a,b]
            h[Pixel[0],Pixel[1],Pixel[2]] += 1
    #3
    RP = np.unravel_index(np.argmax(h),  (256, 256, 256))
    RP = np.asarray(RP, dtype='unit8')
    return RP

In order to extract the most frequently used pixel we will create a three dimentional matrix using np.zero and count how frequently each color appears by adding 1 each time each color appears. #1 is the code that creates the three dimentional matrix of zeros.

Step2 is the same as the code we wrote to add every pixel in a frame to a list. This time, we will be reading the color values of each pixel and add 1 to the corresponding point in the matrix.

In #3, np.argmax(h) is the code that returns the index of the maximum value within a matrix. np.unravel_index changes the argmax value which is presented as a flat type data into tuple form. In order to change the tuple form data into an numpy array, we have to change the value type with np.asarray.

![image](MovieBarcode-WildFlower(Freq).png)

__Conclusion__

In this report we have learned how to create various movie barcodes by using different ways to choose representative colors. What was surprising was that all we had to do to create different barcodes was change the UDF and nothing else. However, although various ways of creating barcodes offer us a wider range of choice, some of the codes are too complex and require way too much data processing to be executed in an average computer.